<a href="https://colab.research.google.com/github/torix/Lecture-ComplexNetworks/blob/master/ComplexNetwork02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ネットワーク指標の計算

NetworkXを使って様々なネットワークの指標を計算します．

NetworkXのドキュメントは，https://networkx.github.io/documentation/stable/index.html を参考．


# 0．準備

## 0.1. 日本語の表示

一番最初にGoogleColaboratory上で図に日本語を利用するためのおまじないを実行します．


以下のコードを実行して，出力された値を次のコードに書いて実行します．<br>
最初に一度だけ行ってください．

In [ ]:
!pip install japanize-matplotlib

!apt-get -y install fonts-ipafont-gothic
!rm -r /root/.cache/matplotlib

ここまで実行後，一度ランタイムをリブートします．

一度実行後は，ここまでのコードは実行する必要がありません．




## 0.1. NetworkXの準備

ネットワークのライブラリであるNetworkXと，描画用にmatplotlibの中のpyplotをインポートします．<br>
%matplotlib inlineはjupyter内でグラフを表示するためのおまじないです．

ついでに，ネットワーク読み込み用の関数もここで定義しておきます．





In [ ]:
import matplotlib.pyplot as plt
import japanize_matplotlib 

import networkx as nx
from IPython.display import set_matplotlib_formats
%matplotlib inline

import requests
from google.colab import files

def readNetwork(url=None, splitter=","):
  # ネットワークを読み込む
  if url == None:
    uploaded = files.upload()
    edge_text = list(uploaded.values())[0].decode()
  else:
    res = requests.get(url)
    res.encoding = res.apparent_encoding
    edge_text = res.text

  # edgeとして読み込める形式に変換
  edge_list = list(item.split(splitter) for item in edge_text.split("\n"))
  edge_list = [a for a in edge_list if len(a) != 1]
  # 無向グラフを作成
  G = nx.Graph() 

  # ネットワークの作成
  G.add_edges_from(edge_list)
  return G

def showNetwork(G, node_value=None):
  #描画する
  # 描画サイズ
  plt.figure(figsize=(10,10))
  # ノードをバネモデルで配置する
  pos = nx.spring_layout(G, k=0.3, seed=0)
  # ノード，エッジ，ラベルの描画
  nx.draw_networkx_edges(G, pos, edge_color='red', width=1.5)
  if node_value == None:
    nx.draw_networkx_labels(G, pos)
    nx.draw_networkx_nodes(G, pos, node_color='white', edgecolors="black")
  else:
    nx.draw_networkx_labels(G, pos)
    nx.draw_networkx_nodes(G, pos, node_color=list(node_value.values()), cmap=plt.cm.Reds, edgecolors="black")
  
  # 描画
  plt.show()


G = readNetwork("http://syrinx.q.t.u-tokyo.ac.jp/tori/lecture/complexnetworks/karate.dat")
showNetwork(G)


# 1. ネットワーク指標の計算

## 1.1. 次数
### 1.1.1．平均次数

各ノードが持つノードの次数の平均を平均次数と言います．

おおよそどのくらいリンクがあるのかをみるための指標としてよく使われます．




In [ ]:
from statistics import mean, median,variance,stdev

degree = G.degree #次数を取得
d_ary = dict(degree).values() #次数のリスト
print(mean(d_ary)) #平均
print(median(d_ary)) #中央値
print(variance(d_ary)) #分散
print(stdev(d_ary)) #標準偏差
print(max(d_ary)) #最大値
print(min(d_ary)) #最小値




### 1.1.2 次数分布


次数がどの程度のノードがどのくらい存在するのかを求めたものが次数分布です．









In [ ]:
import collections

# 次数分布を取得
degree = G.degree
degree_distribution = collections.Counter(dict(degree).values()) # 次数を次数分布のヒストグラムに変換
print("次数の分布")
print(sorted(degree_distribution.items(), key=lambda x:x[0]))


# 次数分布をプロットする
plt.plot(list(degree_distribution.keys()), list(degree_distribution.values()), 'bo') #次数分布をプロット

# 両対数グラフをやめる場合は，この二つのコメントアウトをします．
plt.xscale('log') 
plt.yscale('log')


plt.show()

## 1.2. ネットワークの凝集性

### 1.2.1. リンク密度

完全グラフ（すべてのノードがお互いに隣接している状態）を1としたときに，
実際にリンクが張られている割合を示したものがリンク密度です．

ネットワーク内のノードがどのくらい密につながり合っているかを示す指標です．
大域的な凝集性を示します．

### 1.2.2. クラスタ係数
あるノードの隣接ノード同士が隣接している割合を示す係数です．
局所的な凝集性を示します．




In [ ]:
# ネットワーク密度
print(nx.density(G))

#クラスタ係数
print(nx.average_clustering(G)) #クラスタ係数の平均
clustering_coefficient = nx.clustering(G) 
print(dict(clustering_coefficient)) #ノードごとのクラスタ係数


## 1.2．ネットワークの大きさ

ネットワークの大きさを示す指標として，

*   平均経路長
*   直径

などがあります．




In [ ]:
print("平均経路長")
print(nx.average_shortest_path_length(G)) #平均経路長
print("直径")
print(nx.diameter(G)) #直径


## 1.3. 次数相関

次数相関は，隣接するノードの次数の相関を表します．

次数相関がプラスの場合，次数が高いノードは次数が高いノードと，次数が低いノードは次数が低いノードとつながりがちであることを意味します．

逆に次数相関がマイナスだと，次数が高いノードは低いノードと隣接する可能性が高いことを意味します．

一般に自然界で作られるネットワークでは次数相関はマイナスとなり，人工的なネットワークでは次数相関がプラスになりやすいと言われています．


In [ ]:
print("次数相関")
nx.degree_assortativity_coefficient(G)

## 1.4. 中心性

### 1.4.1.次数中心性
どの程度次数を持っているかによってノードの中心性を評価する指標です．

次数の値そのものを利用します．

ハブノードの中心性が高くなります．




In [ ]:
# degre_centrality の計算
pr = nx.pagerank(G)
degree_centralities = nx.degree_centrality(G)

showNetwork(G, degree_centralities)



### 1.4.2. 近接中心性

他のノードへの距離が近いノードほど高い値を示す中心性指標です．
ネットワークの中央にいるのか，周辺にいるのかが分かります．

すべての他のノードからの距離の逆数によって算出します．


In [ ]:

showNetwork(G, nx.closeness_centrality(G))




### 1.4.3. 媒介中心性

ネットワーク全体の接続性を確保するためにどのくらい重要なノードかを示す指標です．

任意の二つのノードの最短経路上に存在する可能性が高ければ高いほど高い値を示します．



In [ ]:
showNetwork(G, nx.betweenness_centrality(G))

# バーベルグラフによるBetweeness
barbellG = readNetwork("http://syrinx.q.t.u-tokyo.ac.jp/tori/lecture/complexnetworks/barbell-graph.dat")
showNetwork(barbellG, nx.betweenness_centrality(barbellG))


### 1.4.4. ページランク

ページランクは，「リンクがより多く集まっているWEBページはより重要である」という考えに基づいて
Googleの創設者の一人であるラリーペイジが開発した中心性手法です．




In [ ]:
showNetwork(G, nx.pagerank(G))

